In [1]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji
%load_ext autoreload
%autoreload 2

In [70]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import requests
from datetime import date

with open ("config.json", "r") as j:
    config = json.load(j)
with open ("context.json", "r") as j:
    context = json.load(j)

In [3]:
try:
    gc = girder_client.GirderClient(
        apiUrl="".join([
            "http://",
            config["girder"]["host"],
            "/api/v1"
        ])
    )
    gc.authenticate(
        config["girder"]["user"],
        config["girder"]["password"])
    print("Connected to the Girder database 🏗🍃")
except:
    print(
        "I am unable to connect to the "
        "Girder database 🏗🍃"
    )

Connected to the Girder database 🏗🍃


In [4]:
activities_id = postgres_to_girder.get_girder_id_by_name(
    entity="collection",
    name="activities",
    girder_connection=gc
)

In [5]:
try:
    conn = psycopg2.connect(
        " ".join(
            [
                "=".join([
                    key,
                    config["postgres"][key]
                ]) for key in config["postgres"]
            ]
        )
    )
    print("Connected to the Postgres database 🐘")
except:
    print(
        "I am unable to connect to the "
        "Postgres database 🐘"
    )

Connected to the Postgres database 🐘


In [6]:
acts = pd.io.sql.read_sql_query(
    "SELECT * FROM acts;",
    conn
)
users = pd.io.sql.read_sql_query(
    "SELECT * FROM users;",
    conn
)

Pull respondents out of titles in DataFrame from Postgres:

In [7]:
acts["Respondent"] = acts["title"].apply(
    lambda x: x.split(
        " - "
    )[
        1
    ].split(
        " "
    )[
        0
    ] if " - " in x else x.split(
        " – "
    )[
        1
    ].split(
        " "
    )[
        0
    ] if " – " in x else x.split(
        "-"
    )[
        1
    ].split(
        " "
    )[
        0
    ] if "Scale-" in x else x.split(
        " ― "
    )[
        1
    ].split(
        "-"
    )[
        0
    ] if "―" in x else x.split(
        "-"
    )[
        1
    ].split(
        ")"
    )[
        0
    ] if "Index-" in x else "Self" if (
        (
            "_SR" in x
        ) or (
            "-SR" in x
        )
    ) else "Parent" if (
        "_P" in x
    ) else ""
)
acts["title"] = acts["title"].apply(
    lambda x: x.split(
        " - "
    )[
        0
    ] if " - " in x else x.split(
        " – "
    )[
        0
    ] if " – " in x else x.split(
        "-"
    )[
        0
    ] if "Scale-" in x else x.split(
        " ― "
    )[
        0
    ] if "―" in x else x.split(
        "-"
    )[
        0
    ] if "Index-" in x else x.replace(
        " Self Report",
        ""
    ).replace(
        " Parent Report",
        ""
    )
).apply(
    lambda x: "{0})".format(
        x
    ) if (
        "(" in x
    ) and (
        ")"
    ) not in x else x
)

In [48]:
acts

,id,title,type,user_id,act_data,created_at,updated_at,status,organization_id,Respondent
0,1,EHQ (Edinburgh Handedness Questionnaire),survey,1,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-15 08:36:22.382000-05:00,2018-02-15 08:36:22.382000-05:00,active,None,
1,2,CPIC (Children's Perception of Interparental C...,survey,1,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-15 08:36:22.646000-05:00,2018-02-15 08:36:22.646000-05:00,active,None,
2,3,APQ_SR (Alabama Parenting Questionnaire),survey,1,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-15 08:36:22.652000-05:00,2018-02-15 08:36:22.652000-05:00,active,None,Self
3,4,Daily Diary Card (CMI DBT Diary Card),survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-19 15:07:49.975000-05:00,2018-02-19 15:07:49.975000-05:00,active,None,
4,5,Daily Diary Card (CMI DBT Diary Card),survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-19 15:07:54.157000-05:00,2018-02-19 15:07:54.157000-05:00,active,None,
5,6,Daily Diary Card (CMI DBT Diary Card),survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-19 17:18:18.046000-05:00,2018-02-19 17:18:18.046000-05:00,active,None,
6,33,Good things,voice,6,"{""frequency"":""1d"",""timer"":0,""instruction"":""Tal...",2018-02-21 17:51:11.768000-05:00,2018-02-21 17:51:46.780000-05:00,active,None,
7,31,What's he looking at?,drawing,6,"{""frequency"":""1d"",""timer"":10,""image_url"":""http...",2018-02-21 17:49:21.721000-05:00,2018-02-21 17:53:31.092000-05:00,active,None,
8,10,Example survey,survey,6,"{""questions"":[{""type"":""multi_sel"",""rows"":[{""te...",2018-02-21 12:00:58.491000-05:00,2018-02-21 12:08:48.687000-05:00,active,None,
9,16,PAQ_C (Physical Activity Questionnaire for Old...,survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-21 17:48:20.234000-05:00,2018-02-21 17:48:20.234000-05:00,active,None,


In [71]:
for i, activity in enumerate(
    list(
        acts["title"]
    )
):
    abbreviation = None
    if "(" in activity:
        anames = [
            a.strip(
                ")"
            ).strip() for a in activity.split(
                "("
            )
        ]
        if (
            len(anames)==2
        ):
            if (
                len(anames[0])>len(anames[1])
            ):
                abbreviation = anames[1]
                activity_name = anames[0]
            else:
                abbreviation = anames[0]
                activity_name = anames[1]
        else:
            print(anames)
            
    respondent = acts[
        acts["title"]==activity
    ]["Respondent"].values[0]
    item_version = "{0} ({1})".format(
        " ― ".join([
            "Healthy Brain Network",
            "{0} ({1}) {2}".format(
                activity_name,
                abbreviation,
                "― {0} Report".format(
                    respondent
                ) if len(respondent) else ""
            ) if abbreviation else "{0} {1}".format(
                activity,
                "― {0} Report".format(
                    respondent
                ) if len(respondent) else ""
            )
        ]).strip(" "),
        date.strftime(
            acts.loc[
                i,
                "updated_at"
            ],
            "%F"
        )
    )
        
    # Create or locate top-level folder and return _id
    activity_folder_id = gc.createFolder(
        name=activity if not abbreviation else activity_name,
        parentId=activities_id,
        parentType="collection",
        public=True,
        reuseExisting=True
    )["_id"]
    
    # Create or locate item
    activity_item_id = gc.createItem(
        name=item_version,
        parentFolderId=activity_folder_id,
        reuseExisting=True,
        metadata={
            **context,
            "schema:name": {
                "@value": activity if not abbreviation else activity_name,
                "@language": "en-US"
            },
            "abbreviation": abbreviation if abbreviation else None,
            "@type": acts.loc[
                i,
                "type"
            ],
            "status": acts.loc[
                i,
                "status"
            ],
            "pav:lastUpdatedOn": acts.loc[
                i,
                "updated_at"
            ].isoformat()
        }
    )["_id"]

In [62]:
context

{'@context': {'@base': 'http://localhost:8080/api/v1/',
  'nidm': 'http://nidm.nidash.org/specs/nidm-experiment.html#',
  'oslc': 'http://open-services.net/ns/core#',
  'pav': 'http://purl.org/pav/',
  'pav:lastUpdatedOn': {'@type': 'xsd:dateTime'},
  'schema': 'http://schema.org/',
  'schema:description': {'@type': 'xsd:string'},
  'schema:name': {'@type': 'xsd:string'},
  'xsd': 'http://www.w3.org/2001/XMLSchema'}}